**SA305 &#x25aa; Linear Programming &#x25aa; Spring 2021 &#x25aa; Uhan**

# Project. The Diet Problem

⚠️ In order to complete this project, you need to have Pyomo and GLPK installed on your computer.

## Introduction

The goal of the __diet problem__ is to find the cheapest combination of foods that will satisfy a collection of daily nutritional requirements. In particular, we are given a set of foods and a set of nutrients. We are also given the cost per serving for each food, and the minimum and maximum daily requirements for each nutrient. In addition, we have the amount of each nutrient in a serving of each food.

Some history about the diet problem [[source]](https://neos-guide.org/content/diet-problem):

> The diet problem was one of the first optimization problems studied in the 1930s and 1940s. The problem was motivated by the Army's desire to minimize the cost of feeding GIs in the field while still providing a healthy diet. One of the early researchers to study the problem was George Stigler, who made an educated guess of an optimal solution using a heuristic method. His guess for the cost of an optimal diet was \\$39.93 per year (1939 prices). In the fall of 1947, Jack Laderman of the Mathematical Tables Project of the National Bureau of Standards used the newly developed simplex method to solve Stigler's model. As the first "large scale" computation in optimization, the linear program consisted of nine equations in 77 unknowns. It took nine clerks using hand-operated desk calculators 120 man days to solve for the optimal solution of \\$39.69. Stigler's guess was off by only \\$0.24 per year!

## A parameterized model for the diet problem

__Sets.__
\begin{align*}
F & = \mbox{set of foods}\\
N & = \mbox{set of nutrients}\\
\end{align*}

__Parameters.__
\begin{align*}
c_j & = \mbox{cost of one serving of food $j$} && \text{for } j \in F\\
t_j & = \mbox{type of food $j$} && \text{for } j \in F\\
l_i & = \mbox{minimum daily requirement for nutrient $i$} && \text{for } i \in N\\
u_i & = \mbox{maximum daily requirement for nutrient $i$} && \text{for } i \in N\\
a_{ij} & = \mbox{amount of nutrient $i$ in one serving of food $j$} && \text{for } i \in N, j \in F
\end{align*}

__Decision variables.__
\begin{equation*}
x_j = \mbox{number of servings of food $j$} \quad \text{for } j \in F
\end{equation*}

__Objective function and constraints.__
\begin{array}{llll}
\min & \displaystyle\sum_{j \in F} c_j x_j & & \mbox{ (total cost) }\\
\mbox{s.t.}&  l_i \leq \displaystyle\sum_{j \in F} a_{ij}x_j \leq u_i & \text{for } i \in N & \mbox{ (nutritional requirements) }\\
 & x_j \geq 0 & \text{for } j \in F &
\end{array}

Note that the parameter $t_j$ is *not* used in the formulation above, but we will use it below.

---

## Reading and processing data

Reading and processing data is often a significant portion of the work necessary to get an optimization model up and running. In this project, you will learn one way to approach these tasks.

### Reading data with pandas

Typically, data is stored in a file such as a spreadsheet or comma separated value (CSV) file. There are many ways to read in such files using Python. 

For this project, you will use a library called [Pandas](https://pandas.pydata.org/). The main object in Pandas is the __DataFrame__, which is a two-dimensional table, with rows and columns.

In the same folder as this notebook, there is an Excel file called `mcdonalds.xlsx`. Before you go on, open this file and familiarize yourself with the three worksheets contained in that file. It's often useful to look at the actual raw data before processing it.

We start by importing Pandas. Next, we use the Pandas method `read_excel` to read the `Food` worksheet in `mcdonalds.xlsx` into a Pandas DataFrame called `food_df`. Then, we use the DataFrame method `head` to display the first 5 rows of `food_df`.

In [ ]:
import pandas as pd

food_df = pd.read_excel('mcdonalds.xlsx', sheet_name='Food', index_col='Food')

food_df.head()

Pandas assigns a **label** to each row/observation of a DataFrame, based on the input data. These labels are called the **index**. In our code above, we used the keyword argument `index_col='Food'` to tell Python which column of the worksheet to use as the index. In the output above, you can see the labels in boldface at the left of each row.

### Processing data 

Pandas has handy functions to create the lists and dictionaries we need to formulate our optimization models in Pyomo. 

For example, here is code to create a list for the set of foods $F$ and a dictionary for the food costs $c_j$ for $j \in F$.

In [ ]:
food_list = food_df.index.to_list()

food_list

In [ ]:
cost_dict = food_df.to_dict()['Cost']

cost_dict

Sometimes, however, we need to work harder to process the data. For example, consider the tabular data in the `FoodNutrients` worksheet of `mcdonalds.xlsx`. We can read it in as follows:

In [ ]:
food_nut_df = pd.read_excel('mcdonalds.xlsx', sheet_name='FoodNutrients', index_col='Food')

food_nut_dict_raw = food_nut_df.to_dict()

food_nut_dict_raw

If you look carefully, you'll see that `food_nut_dict_raw` is actually a dictionary of dictionaries, or a nested dictionary. The information in the nested dictionary `food_nut_dict_raw` gives us the information we need for the parameter $a_{ij}$ for $i \in N$ and $j \in F$. However, this dictionary isn't in the form that Python requires.

Let's illustrate this concretely. To get the dictionary that contains calorie information, we write

In [ ]:
food_nut_dict_raw['Cal']

So, to get the calories in a Cheeseburger from `foot_nut_dict_raw`, we write

In [ ]:
food_nut_dict_raw['Cal']['Cheeseburger']

However, to model the parameter $a_{ij}$, Pyomo needs a dictionary whose keys are _tuples_: in particular, of the form `(i, j)`, where `i` corresponds to a nutrient, and `j` corresponds to a food. In other words, we want a dictionary where we would write the following to get the calories in a Cheeseburger:

```python
food_nut_dict[('Cal','Cheeseburger')]
```

We can convert the "raw" nested dictionary into the form we need for Pyomo by using a __dictionary comprehension__, which is a short code block that creates a dictionary from other data structures using for statements and conditionals. Consider the following example.

In [ ]:
dict_of_dicts = {
    'A': {'first': 1, 'second': 2, 'third': 3},
    'B': {'first': 11, 'second': 12, 'third': 13}
}

tuple_dict = {
    (i, j): dict_of_dicts[i][j] 
    for i in dict_of_dicts.keys() 
    for j in dict_of_dicts[i].keys()
}

tuple_dict

---

## Your assignment

__1.__ Using Pandas, read in the data in `mcdonalds.xlsx` into lists and dictionaries that define values for the sets and parameters in the parameterized model above.

You will need:

- a list of foods
- a list of nutrients
- a dictionary that maps foods to their types
- a dictionary that maps nutrients to their minimum daily requirement
- a dictionary that maps nutrients to their maximum daily requirement
- a dictionary that maps each pair (nutrient, food) to the amount of nutrient in that food

__2.__ Implement the parameterized model above in Pyomo. Use your answer from the previous question and code from Lab 2 as a template.

__3.__ Solve the model. Output the solution.

In particular, write code that invokes `glpk` to solve the linear program. For the solution, only print decision variables that are positive in an optimal solution.

__4.__ Interpret and critique the solution. Ignore integrality issues and focus on the composition of the menu.

*Write your answer here. Double-click to edit.*

__5.__ Let's create a more palatable diet, using the food types.

For example, suppose we want to ensure that there are at most 6 condiments in our chosen diet. We can write such a constraint like this:

\begin{equation*}
\sum_{j \in F: t_j = \text{Condiment}} x_j \le 6
\end{equation*}

Recall from Lab 3 that we can write such a constraint with a sum using ":" notation in Pyomo like this:

```python
def condiment_rule(model):
    return sum(model.x[j] for j in model.F if model.food_type[j] == 'Condiment') <= 6

model.condiment = pyo.Constraint(rule=condiment_rule)
```

Do the following to create a more palatable diet:

- Add the food types parameter $t_j$ for $j \in F$ to your model.
- Add constraints using this parameter to enforce the following:
    - There are at most 6 condiments in the diet.
    - There are at least 2 main dishes in the diet.
    - There is at least 1 breakfast dish in the diet.
    - There are at least 3 beverages in the diet.

⚠️ When defining the food types as a Pyomo parameter, use the keyword argument `within=pyo.Any` in `pyo.Param`, to let Pyomo know that this parameter is not numeric.

__6__. Solve the new model. Output the solution.

__7.__ Create your own minimum cost diet.

__a.__ In the Markdown cell below, in words, describe at least 3 side constraints that modify the original solution from __#3__ so that it suits conditions you would want in your daily diet. 

*Write your answer here. Double-click to edit.*

b. in the code cell below, write a new model for the diet problem in Pyomo, adding your own side constraints from your answer to __#7a__. __Start from scratch so that the constraints you added in #5 are not included.__ Solve your new model and output the solution.

In [ ]:
# Put your new model here


---

## Grading rubric

| Problem | Points |
| :-: | -: |
| 1 | 10 |
| 2 | 15 |
| 3 | 5 |
| 4 | 10 |
| 5 | 15 |
| 6 | 5 |
| 7a | 10 |
| 7b | 20 |
| __Total__ | __90 points__ |